In [11]:
# to do
def get_edges_list_with_labels(tweets):
    return [('"t/test-01"', '"s/system"', '":ARG1"'),
 ('"t/test-01"', '"t2/this"', '":domain"')]



!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-geometric
import torch
from torch_geometric.data import Data

Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html


In [13]:
class DataParser:
    edge_label_to_index = {}
    word_to_index = {}


    def build_dictionaries(self, tweets_list):
        for tweet in tweets_list:
            edges_list = get_edges_list_with_labels(tweet)
            for edge in edges_list:
                source = edge[0].split('/')[1]
                target = edge[1].split('/')[1]
                edge_label = edge[2]
                if source not in self.word_to_index:
                    self.word_to_index[source] = len(self.word_to_index)
                if target not in self.word_to_index:
                    self.word_to_index[target] = len(self.word_to_index)
                if edge_label not in self.edge_label_to_index:
                    self.edge_label_to_index[edge_label] = len(self.edge_label_to_index)


    def build_data_graph(self, tweet, target):
        word_to_node_number = {}
        edges_list = get_edges_list_with_labels(tweet)
        sources = [0] * len(edges_list)
        targets = [0] * len(edges_list)
        edge_index = [sources, targets]
        edge_attr = []
        x = [] 
        i = 0

        for edge in edges_list:

            source = edge[0].split('/')[0]
            source_word = edge[0].split('/')[1]
            target = edge[1].split('/')[0]
            target_word = edge[1].split('/')[1]

            if source not in word_to_node_number:

                word_to_node_number[source] = len(word_to_node_number)
                
                # one hot vector representing the node word
                word_vector = [0] * len(self.word_to_index)
                word_vector[self.word_to_index[source_word]] = 1
                x.append(word_vector)

            if target not in word_to_node_number:
                word_to_node_number[target] = len(word_to_node_number)
                
                # one hot vector representing the node word
                word_vector = [0] * len(self.word_to_index)
                word_vector[self.word_to_index[target_word]] = 1
                x.append(word_vector)

            edge_label = edge[2]

            # build the list of edges
            sources[i] = word_to_node_number[source]
            targets[i] = word_to_node_number[target]
            
            # one hot vector representing the type of relation between  source and target node
            edge_vector = [0] * len(self.edge_label_to_index)
            edge_vector[self.edge_label_to_index[edge_label]] = 1
            edge_attr.append(edge_vector)


            i += 1

            
        print(edge_index)
        print(edge_attr)
        print(x)
        
    
        edge_index = torch.tensor([[0, 1, 1, 2],
                                   [1, 0, 2, 1]], dtype=torch.long)
        x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
        
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=target)
        return data


In [15]:
DP = DataParser()
DP.build_dictionaries([""])
print(DP.word_to_index)
print(DP.edge_label_to_index)
DP.build_data_graph("", "positive")


{'test-01"': 0, 'system"': 1, 'this"': 2}
{'":ARG1"': 0, '":domain"': 1}
[[0, 0], [1, 2]]
[[1, 0], [0, 1]]
[[1, 0, 0], [0, 1, 0], [0, 0, 1]]


Data(x=[3, 1], edge_index=[2, 4], edge_attr=[2], y='"t2')